In [ ]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---

import pandas as pd
import numpy as np
import joblib 
import os 

# Bibliotecas de Pré-processamento e Métricas
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
# Modelos Non_Linear para classificação
from sklearn.neighbors import KNeighborsClassifier # KNN para Classificação
# Modelos de Ensemble para Classificação
from sklearn.ensemble import RandomForestClassifier # Random Forest
import xgboost as xgb # Gradient Boosting (XGBoost)

print("Bibliotecas importadas e prontas para Classificação NonLinear and Ensemble (Pima Indians Diabetes).")

Bibliotecas importadas e prontas para Classificação Ensemble (Pima Indians Diabetes).


In [2]:
# --- CÉLULA 2: Carregamento de Dados Pima (Arquivos Salvos no Notebook 03) ---

DATA_DIR = 'data'

# Carregamento dos conjuntos padronizados e separados 
try:
    X_train_scaled = joblib.load(os.path.join(DATA_DIR, 'X_train_pima_scaled.joblib'))
    X_test_scaled = joblib.load(os.path.join(DATA_DIR, 'X_test_pima_scaled.joblib'))
    y_train = joblib.load(os.path.join(DATA_DIR, 'y_train_pima.joblib'))
    y_test = joblib.load(os.path.join(DATA_DIR, 'y_test_pima.joblib'))
    print("Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.")
except Exception as e:
    # Se os arquivos não existirem, será necessário rodar a Célula 2 do Notebook 03
    print(f"ERRO: Não foi possível carregar os arquivos. Verifique se a Célula 2 do Notebook 03 foi executada. Erro: {e}")

print(f"Shape do X_train: {X_train_scaled.shape}")

Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.
Shape do X_train: (537, 8)


In [7]:
# --- CÉLULA 3: KNN Classifier ---

# 1. Definir o range de hiperparâmetros (Número de Vizinhos K)
param_grid_knn = {
    'n_neighbors': [5, 7, 11, 15, 21], # Valores comuns de K
    'weights': ['uniform', 'distance'] # Diferentes formas de ponderar os vizinhos
}

# 2. Configurar o GridSearchCV
knn = KNeighborsClassifier()
knn_grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid_knn,
    scoring='roc_auc', # Otimizamos para AUC-ROC
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
# O KNN é altamente sensível à escala, por isso é crucial usar os dados SCALED
knn_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_knn_model = knn_grid.best_estimator_
MODEL_DIR = 'models'
MODEL_PATH_KNN = os.path.join(MODEL_DIR, 'knn_classifier_pima.joblib')
joblib.dump(best_knn_model, MODEL_PATH_KNN)
print(f"\nModelo KNN Classifier otimizado e salvo em: {MODEL_PATH_KNN}")

# 5. Avaliação
y_pred_proba_knn = best_knn_model.predict_proba(X_test_scaled)[:, 1] # Probabilidade da Classe 1
y_pred_knn = best_knn_model.predict(X_test_scaled) # Previsão da classe

auc_knn = roc_auc_score(y_test, y_pred_proba_knn)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print(f"\nKNN: Melhor Parâmetro encontrado: {knn_grid.best_params_}")
print(f"KNN: Acurácia (Teste): {accuracy_knn:.4f}")
print(f"KNN: AUC-ROC (Teste): {auc_knn:.4f}")
print("\nRelatório de Classificação (KNN):")
print(classification_report(y_test, y_pred_knn))


Modelo KNN Classifier otimizado e salvo em: models\knn_classifier_pima.joblib

KNN: Melhor Parâmetro encontrado: {'n_neighbors': 21, 'weights': 'distance'}
KNN: Acurácia (Teste): 0.7706
KNN: AUC-ROC (Teste): 0.8127

Relatório de Classificação (KNN):
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       150
           1       0.73      0.54      0.62        81

    accuracy                           0.77       231
   macro avg       0.76      0.72      0.73       231
weighted avg       0.77      0.77      0.76       231



In [ ]:
# --- CÉLULA 4: Random Forest Classifier ---

# 1. Definir o range de hiperparâmetros
param_grid_rf = {
    'n_estimators': [100, 200], # Número de árvores
    'max_depth': [5, 10],       # Profundidade máxima
    'min_samples_split': [5, 10]
}

# 2. Configurar o GridSearchCV
rf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='roc_auc', # Otimizamos para AUC-ROC
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
rf_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_rf_model = rf_grid.best_estimator_
MODEL_DIR = 'models'
MODEL_PATH_RF = os.path.join(MODEL_DIR, 'random_forest_classifier_pima.joblib')
joblib.dump(best_rf_model, MODEL_PATH_RF)
print(f"\nModelo Random Forest Classifier otimizado e salvo em: {MODEL_PATH_RF}")

# 5. Avaliação
y_pred_proba_rf = best_rf_model.predict_proba(X_test_scaled)[:, 1] # Probabilidade da Classe 1
y_pred_rf = best_rf_model.predict(X_test_scaled) # Previsão da classe

auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"\nRandom Forest: Melhor Parâmetro encontrado: {rf_grid.best_params_}")
print(f"Random Forest: Acurácia (Teste): {accuracy_rf:.4f}")
print(f"Random Forest: AUC-ROC (Teste): {auc_rf:.4f}")
print("\nRelatório de Classificação (Random Forest):")
print(classification_report(y_test, y_pred_rf))


Modelo Random Forest Classifier otimizado e salvo em: models\random_forest_classifier_pima.joblib

Random Forest: Melhor Parâmetro encontrado: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Random Forest: Acurácia (Teste): 0.7359
Random Forest: AUC-ROC (Teste): 0.8305

Relatório de Classificação (Random Forest):
              precision    recall  f1-score   support

           0       0.76      0.87      0.81       150
           1       0.67      0.49      0.57        81

    accuracy                           0.74       231
   macro avg       0.71      0.68      0.69       231
weighted avg       0.73      0.74      0.72       231



In [ ]:
# --- CÉLULA 5: XGBoost Classifier (Gradient Boosting) ---

# 1. Definir o range de hiperparâmetros
param_grid_xgb = {
    'n_estimators': [100, 200],      
    'max_depth': [3, 5],            
    'learning_rate': [0.05, 0.1]     
}

# 2. Configurar o GridSearchCV
# Usamos 'objective='binary:logistic' para problemas de classificação binária
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, random_state=42)
xgb_grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='roc_auc', # Otimizamos para AUC-ROC
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
xgb_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_xgb_model = xgb_grid.best_estimator_
MODEL_DIR = 'models'
MODEL_PATH_XGB = os.path.join(MODEL_DIR, 'xgboost_classifier_pima.joblib')
joblib.dump(best_xgb_model, MODEL_PATH_XGB)
print(f"\nModelo XGBoost Classifier otimizado e salvo em: {MODEL_PATH_XGB}")

# 5. Avaliação
y_pred_proba_xgb = best_xgb_model.predict_proba(X_test_scaled)[:, 1] # Probabilidade da Classe 1
y_pred_xgb = best_xgb_model.predict(X_test_scaled) # Previsão da classe

auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"\nXGBoost: Melhor Parâmetro encontrado: {xgb_grid.best_params_}")
print(f"XGBoost: Acurácia (Teste): {accuracy_xgb:.4f}")
print(f"XGBoost: AUC-ROC (Teste): {auc_xgb:.4f}")
print("\nRelatório de Classificação (XGBoost):")
print(classification_report(y_test, y_pred_xgb))


Modelo XGBoost Classifier otimizado e salvo em: models\xgboost_classifier_pima.joblib

XGBoost: Melhor Parâmetro encontrado: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
XGBoost: Acurácia (Teste): 0.7576
XGBoost: AUC-ROC (Teste): 0.8416

Relatório de Classificação (XGBoost):
              precision    recall  f1-score   support

           0       0.78      0.87      0.82       150
           1       0.70      0.54      0.61        81

    accuracy                           0.76       231
   macro avg       0.74      0.71      0.72       231
weighted avg       0.75      0.76      0.75       231



c:\Users\RonaldoRosarioRamos\miniconda3\envs\python_modelos_predicao_estatistica\Lib\site-packages\xgboost\training.py:199: UserWarning: [15:24:41] WARNING: D:\bld\xgboost-split_1764148459144\work\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
